In [2]:
# импортируем библиотеки
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.corpus import stopwords
import pymorphy2
import re

# забираем данные

In [3]:
# путь к файлам
path = '/data/share/lab05data/'

In [4]:
# текст для классификации
id_text = [2561, 2049, 4, 2565, 2562, 2062, 3600, 3603, 3588, 534, 3607, 24, 25, 3612, 3102, 2591, 3804, 3627, 1580, 46, 1074, 2100, 3125, 56, 1593, 2485, 2621, 63, 3139, 2316, 610, 2121, 1098, 1099, 588, 2126, 2639, 88, 2140, 607, 1634, 2067, 612, 3175, 3178, 531, 1647, 1136, 1137, 1139, 1146, 1151, 22, 1710, 2692, 1160, 1674, 1560, 3729, 623, 3224, 153, 2206, 1183, 160, 3235, 3748, 684, 1651, 1711, 30, 1714, 2079, 695, 2232, 698, 189, 1227, 1025, 711, 3784, 2253, 2763, 3788, 1741, 2255, 1232, 3283, 383, 2775, 2265, 1829, 1244, 3300, 1767, 1257, 1320, 1747, 3830, 1152, 1279, 3030, 913, 3332, 1798, 2311, 386, 1289, 3340, 269, 1806, 274, 1425, 789, 2863, 1305, 2332, 800, 1825, 2339, 292, 293, 2855, 3368, 562, 2861, 3886, 2351, 2864, 307, 2356, 822, 1019, 3722, 2101, 3387, 2688, 829, 3904, 833, 2279, 3398, 2887, 336, 337, 3473, 854, 3420, 2398, 3426, 3939, 3940, 357, 2918, 1387, 2924, 879, 3442, 1396, 2421, 2427, 1407, 1345, 2438, 1928, 1419, 2456, 1054, 2030, 1937, 402, 2628, 2968, 409, 1437, 1956, 421, 1963, 943, 2481, 3509, 955, 2635, 2496, 455, 972, 3025, 1188, 3542, 3544, 3550, 482, 3699, 2812, 2534, 3566, 1521, 3570, 1525, 2043]
test_texts = []
for i in id_text:
    f = open(path + 'test_' + str(i) +'.txt', "r")
    test_texts.append(f.read())
t_texts = pd.DataFrame(test_texts, columns = ['t_texts'])

In [29]:
# текст для обучения
id_base_text = list(range(1, 21))
base_texts = []
for i in id_base_text:
    f = open(path + 'base_' + str(i) +'.txt', "r")
    base_texts.append(f.read())
b_texts = pd.DataFrame(base_texts, columns = ['b_texts'])

# стоп-слова

In [35]:
#встроенные
swords = stopwords.words(['english', 'russian'])
#результат оценки статистики встречаемости. Добавлял интуитивно "вручную"
swords = swords + ['знание', 'опыт', 'работы', 'strong', 'работа',
'компания',
'условие',
'quot',
'обязанность',
'возможность',
'работать',
'заработный',
'обучение',
'профессиональный',
'умение',
'наш',
'язык',
'оформление',
'офис',
'рост',
'график',
'задача',
'дать',
'понимание',
'навык',
'карьерный',
'уровень',
'высокий',
'решение',
'клиент',
'участие',
'английский',
'который',
'коллектив',
'принцип',
'продукт',
'хороший',
'официальный',
'контроль',
'свой',
'интересный',
'использование',
'развитие',
'рабочий',
'результат']



# создаем вектор данных

In [36]:
#из лекции функция
GROUPING_SPACE_REGEX = re.compile(r'([^\w]|[+])', re.UNICODE)
def simple_word_tokenize(text, _split=GROUPING_SPACE_REGEX.split):
    return [t for t in _split(text.lower()) if t and not t.isspace()]
m = pymorphy2.MorphAnalyzer()
def token_r(text):
    words = simple_word_tokenize(text)
    return [m.parse(x)[0].normal_form for x in words if len(x) > 3]

cv = CountVectorizer(stop_words = swords, tokenizer=token_r)

css = cv.fit_transform(b_texts.b_texts).toarray()
css2 = cv.transform(t_texts.t_texts).toarray()

In [37]:
# встречаемость слов для пополнения словаря стоп-слов
words = [y[0] for y in sorted(cv.vocabulary_.items(), key=lambda x: x[1])]
c_words = zip(words, sum(css2))
c_words = sorted(c_words, key=lambda x: x[1], reverse=True)
c_words[:25]

# считаем косинусную близость

In [43]:
#суммируем косинусные близости каждого текста из стестовой и 20 обучающих текстов
ilo = []

for j in range(len(css2)):
    lkl = 0
    for i in range(len(css)):
        lkl += cosine_similarity(css[i].reshape(1, -1), css2[j].reshape(1, -1))[0][0]
    ilo.append(lkl)

In [44]:
# порог решения, если выше, то тип определен
thrshld = np.mean(ilo)

In [45]:
# трансформируем результат в датафрейм (просто, лично мне так удобнее обрабатывать)
df_ilo = pd.DataFrame(ilo, columns = ['ilo'])

In [46]:
# объединяем с исходными текстами, чтобы достать индексы
end_calc = pd.concat([t_texts, df_ilo], axis = 1)

In [47]:
end_calc.head()

,t_texts,ilo
0,<p>Мы ищем талантливых и целеустремленных спец...,4.117645
1,<p><strong>Чем придется заниматься:</strong></...,3.646620
2,<p><strong>Обязанности</strong>:</p> <ul> <li>...,4.569716
3,<p>Ищем опытных разработчиков под Android для ...,3.900656
4,<p><strong><em>Работа в Германии</em></strong>...,1.392930


In [53]:
# разделяем на определенные и не определенные.

dict_to_out = {}
defd = []
othr = []

for i in range(len(end_calc)):
    to_ins = id_text[i] #преобразовываем в инт, т.к. json не понимает numpy
    if end_calc.ilo[i] >= thrshld:
        defd.append(to_ins)
    else:
        othr.append(to_ins)
    
dict_to_out = {
    'defined' : sorted(defd),
    'other' : sorted(othr)
}

In [54]:
# выводим в файл
import json

with open('lab05.json', 'w') as outfile:
    json.dump(dict_to_out, outfile)